In [ ]:
#!pip install -q tfp-nightly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from scipy.stats import norm
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, cross_val_predict
from catboost import CatBoostRegressor
import tensorflow as tf
#import tensorflow_probability as tfp
#from tensorflow_probability import edward2 as ed
#tfd = tfp.distributions


sns.set()
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%config InlineBackend.figure_format = 'svg'
np.random.seed(111)
tf.set_random_seed(111)

## Data

TODO: intro

In [ ]:
train = pd.read_csv('../input/train.csv', 
                    usecols=np.arange(1, 7),
                    nrows=200000)
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'],
                                          format='%Y-%m-%d %H:%M:%S %Z')

In [ ]:
train.head()

Let's take a look at how many null values there are in each column.

In [ ]:
print('Column\tPercent Null')
for col in train:
    print(col, 100*train[col].isnull().sum()/train.shape[0])

There are some null values, but a negligible amount of them, so we'll simply remove rows with null values.

In [ ]:
train.dropna(inplace=True)

Have to extract time of day, time of week, time of year, and year, and then drop the original datetime column.

In [ ]:
train['min_of_day'] = (60*train['pickup_datetime'].dt.hour + 
                       train['pickup_datetime'].dt.minute)
train['day_of_week'] = train['pickup_datetime'].dt.dayofweek
train['day_of_year'] = train['pickup_datetime'].dt.dayofyear
train['year'] = train['pickup_datetime'].dt.year
train.drop('pickup_datetime', axis=1, inplace=True)

In [ ]:
train.head()

Let's check the fare amount distributions:

In [ ]:
# Plot distribution of fares
sns.distplot(train['fare_amount'])
plt.show()

It looks like there might be some negative values (which doesn't make any sense, of course!). Let's zoom in on the area around 0.

In [ ]:
# Plot distribution of fares around 0
plt.hist(train['fare_amount'], 
         bins=np.arange(-50, 10), log=True)
plt.show()

Let's remove the datapoints with fares which are suspiciously low, and also rides with suspiciously high fares.

In [ ]:
# Function to remove rows outside range
def clip(df, a, b, col):
    for c in col:
        df = df[(df[c]>a) & (df[c]<b)]
    return df

# Remove rows with outlier fare values
train = clip(train, 1, 200, ['fare_amount'])

Finally, let's check the locations of the pickups and dropoffs.

In [ ]:
# Plot distribution of pickup longitudes
fig, ax = plt.subplots(2, 2)
nyc_lon = -74
nyc_lat = 40.7
ax[0,0].axvline(nyc_lon, linestyle='--', color='k')
ax[0,0].hist(train['pickup_longitude'], bins=50, log=True)
ax[0,0].set_ylabel('Pickup')
ax[1,0].axvline(nyc_lon, linestyle='--', color='k')
ax[1,0].hist(train['dropoff_longitude'], bins=50, log=True)
ax[1,0].set_xlabel('Longitude')
ax[1,0].set_ylabel('Dropoff')
ax[0,1].axvline(nyc_lat, linestyle='--', color='k')
ax[0,1].hist(train['pickup_latitude'], bins=50, log=True)
ax[1,1].axvline(nyc_lat, linestyle='--', color='k')
ax[1,1].hist(train['dropoff_latitude'], bins=50, log=True)
ax[1,1].set_xlabel('Latitude')
plt.show()

There are some outliers, especially at 0.  Let's remove rows with geographical locations outside a reasonable range (near the greater NYC metropolitan area).

In [ ]:
# Remove geographical outliers
train = clip(train,  -75, -72.5,
             ['pickup_longitude', 'dropoff_longitude'])
train = clip(train, 40, 41.5,
             ['pickup_latitude', 'dropoff_latitude'])

And now we have only values which are near NYC:

In [ ]:
# Plot distribution of pickup longitudes
fig, ax = plt.subplots(2, 2)
nyc_lon = -74
nyc_lat = 40.7
ax[0,0].axvline(nyc_lon, linestyle='--', color='k')
ax[0,0].hist(train['pickup_longitude'], bins=50, log=True)
ax[0,0].set_ylabel('Pickup')
ax[1,0].axvline(nyc_lon, linestyle='--', color='k')
ax[1,0].hist(train['dropoff_longitude'], bins=50, log=True)
ax[1,0].set_xlabel('Longitude')
ax[1,0].set_ylabel('Dropoff')
ax[0,1].axvline(nyc_lat, linestyle='--', color='k')
ax[0,1].hist(train['pickup_latitude'], bins=50, log=True)
ax[1,1].axvline(nyc_lat, linestyle='--', color='k')
ax[1,1].hist(train['dropoff_latitude'], bins=50, log=True)
ax[1,1].set_xlabel('Latitude')
plt.show()

## Baseline Model

xgboost

In [ ]:
# Separate in- from dependent variables
x_taxi = train.drop('fare_amount', axis=1)
y_taxi = train['fare_amount']

# Make Mean Absolute Error scorer
mae_scorer = make_scorer(mean_absolute_error)

# Function to print cross-validated mean abs deviation
def cv_mae(regressor, x, y, cv=3, scorer=mae_scorer):
    scores = cross_val_score(regressor, 
                             x, y, cv=cv,
                             scoring=scorer)
    print('MAE:', scores.mean())

How well do we do if we just predict the mean?

In [ ]:
# MAE from predicting just the mean
cv_mae(DummyRegressor(), x_taxi, y_taxi)

And if we just use the distance of the trip as a predictor?

In [ ]:
# Distance between pickup and dropoff locations
dist = np.sqrt(
    np.power(train['pickup_longitude'] -
             train['dropoff_longitude'], 2) + 
    np.power(train['pickup_latitude'] - 
             train['dropoff_latitude'], 2))

# MAE from using just distance as predictor
cv_mae(IsotonicRegression(out_of_bounds='clip'), 
       dist, y_taxi)

And we can do a little better if we use gradient boosted decision trees.

In [ ]:
# Cross-validated MAE w/ CatBoost
cv_mae(CatBoostRegressor(logging_level='Silent'), 
       x_taxi, y_taxi)

## Uncertainty under Heteroskedasticity

What if we now want to predict our uncertainty as to our estimate?

But need to look out for heteroskedasticity!  Explain heteroskedasticity

In [ ]:
# A function to generate heteroskedastic data
def h_func(x):
    return x + np.exp(0.5*x)*np.random.randn(x.shape[0],1)

# Generate heteroskedastic data
N = 1000
xx = np.atleast_2d(np.random.randn(N)).T
yy = h_func(xx).ravel()

# Generate validation data
xx_val = np.atleast_2d(np.linspace(-3, 3, N)).T
yy_val = h_func(xx_val).ravel()

# Plot data we generated
plt.plot(xx, yy, '.')
plt.show()

Is the true data heteroskedastic?

In [ ]:
# Plot distance vs fare
sns.jointplot('Distance', 'Fare Amount',
              pd.DataFrame({'Distance':dist, 
                            'Fare Amount':y_taxi}), 
              kind="hex",
              xlim=(0, 0.4), ylim=(0, 100),
              joint_kws=dict(gridsize=70, bins='log'))

Indeed it looks like the true data is heteroskedastic.  And keep in mind this is only the *distance*, whereas the full dataset has pickup/dropoff location, time of day, time of year, etc.  The y-values could be heteroskedastic as a function of those predictors as well!

How can we account for this uncertainty, especially when it is varying with our independent variables?  One way which can be used with gradient boosted decision trees is to do a quantile regression.

Explain quantile loss, penalizes y_pred>y_true more than y_true>y_pred when quantile<0.5 and vice-versa.  Eqs etc

In [ ]:
# Gradient boosted tree regressors w/ different quantile losses
gbrL = CatBoostRegressor(loss_function='Quantile:alpha=0.025', logging_level='Silent')
gbr = CatBoostRegressor(loss_function='Quantile:alpha=0.5', logging_level='Silent')
gbrH = CatBoostRegressor(loss_function='Quantile:alpha=0.975', logging_level='Silent')

# Using scikit-learn's gradient boosted decision trees
#from sklearn.ensemble import GradientBoostingRegressor
#gbrL = GradientBoostingRegressor(loss='quantile', alpha=0.025)
#gbr = GradientBoostingRegressor(loss='quantile', alpha=0.5)
#gbrH = GradientBoostingRegressor(loss='quantile', alpha=0.975)

# Fit to data
gbrL.fit(xx, yy)
gbr.fit(xx, yy)
gbrH.fit(xx, yy)

# Predict on validation data
y_predL = gbrL.predict(xx_val)
y_pred = gbr.predict(xx_val)
y_predH = gbrH.predict(xx_val)

# Plot predictions over points
plt.figure()
plt.plot(xx_val, yy_val, '.',
         label='Validation data')
plt.fill_between(xx_val.ravel(), y_predL, y_predH,
                 alpha=0.3, facecolor=colors[2],
                 label='95% confidence interval')
plt.plot(xx_val, y_pred, 'k', label='Predicted median')
plt.legend()
plt.show()

To see how well-calibrated the model is, we can check the coverage of the 95% confidence interval (the percentage of y values from the validation dataset falling within our 95% predictive interval).  If the model is well-calibrated, the coverage will be near 95%.

In [ ]:
# Function to compute coverage of predictive interval
def coverage(y, yL, yH):
     return (100 / yL.shape[0] *
             ((y>yL)&(y<yH)).sum())
    
# Compute coverage of the 95% interval
print('Coverage of 95%% predictive interval: %0.1f%%'
      % coverage(yy_val, y_predL, y_predH))

Hmm. OK but not great.  How does it look on the real data?

In [ ]:
# Compute 2.5% and 97.5% predictive intervals
y_predL = cross_val_predict(gbrL, x_taxi, y_taxi)
y_predH = cross_val_predict(gbrH, x_taxi, y_taxi)

# Compute coverage of the 95% interval
print('Coverage of 95%% predictive interval: %0.1f%%'
      % coverage(y_taxi, y_predL, y_predH))

Eek.  Not so great at all :(

We could calibrate our model, or we could use a model which is more explicitly built to capture uncertainty accurately!

## Dual-module Bayesian Neural Network

